In [1]:
%load_ext dotenv
%dotenv

In [14]:
import chromadb
from chromadb.config import Settings
import yandexgpt
import os
import openai

In [15]:
chroma_client = chromadb.HttpClient(host='localhost', 
                                    port="8000", 
                                    settings=Settings(anonymized_telemetry=False))
embeddings = yandexgpt.Embeddings(os.getenv("YANDEX_GPT_KEY"), 
                                  os.getenv("YANDEX_GPT_EMBEDDINGS_URI"))
textGenerationAsync = yandexgpt.TextGenerationAsync(os.getenv("YANDEX_GPT_KEY"), 
                                  os.getenv("YANDEX_GPT_URI"))

openai_client = openai.OpenAI(api_key=os.getenv("CHAT_GPT_KEY"))

In [16]:
collection = chroma_client.get_or_create_collection("intents")

In [18]:
questions = [
    "Доброго времени! пытаюсь объединить товар . почему к данному товару не цепляется товар из яндекс маркета . штрихкод и артикул есть .",
    "Здравствуйте, как можно обновить информацию в карточках импортируя из маркетплейсов? Чтобы только обновилась информация, а не добавлялись новые карточки",
    "был заказ со сбера, но остаток не списался",
    "WB сделали для клиентов возможность отмены заказа в течении часа, остатки при такой отмене возвращаться не будут. На работу интеграции как-то повлияет?",
    "Здравствуйте. Что будет если отправить товар размера который не соответствует тому что написано на карточке и который клиент не заказывал. Получается на карточке 2.6 метров, клиент его заказал, но мы отправим 3 метровый",
    "Подскажите, а интеграция с КазанЭкспресс есть у вас?",
    "Доброе время суток! Я хочу заказать услугу настройки Selsup по акции \"Быстрый старт\".",
    "добрый вечер! как обнулить остатки на ВБ на праздники и чтобы они не подгружались в ВБ до определенного дня?",
    "Здравствуйте. А как посмотреть ошибки, которые возникли при импорте на сбер (обновляли связи), выдал 15 ошибок",
    "Здравствуйте. А можно нам оставить на новых условиях тарифа сбер вместо вайлдбериз (мы его не используем), а на сбер потратили кучу времени с вами, его настраивая, будет жалко потерять и если нет, как нам правильно отключить его от сервиса?",
    "Здравствуйте! Подскажите, пожалуйста, по срокам заполнения шк🙏",
    "А почему заказы не собираются автоматические?",
    
]
SYSTEM_PROMPT = ("Ты специалист технической поддержки. "
                 "На основе сообщений, написанных тобой выше, сгенерируй сообщение")

In [21]:
def format_qa(func):
    def wrapper_format_qa(*args, **kwargs):
        print("Вопрос:", args[0])
        print("Ответ:", func(*args, **kwargs))
    return wrapper_format_qa

@format_qa
def generate_yandexgpt_answer(question: str):
    result = collection.query(
        query_embeddings=[embeddings.text_embedding(question)["embedding"]],
        n_results=6,
    )
    
    messages = [yandexgpt.Message(yandexgpt.MessageRole.SYSTEM, SYSTEM_PROMPT)]
    for distance, metadata, document in zip(
            result["distances"][0], result["metadatas"][0], result["documents"][0]
    ):
        if distance < 1:
            messages.append(yandexgpt.Message(yandexgpt.MessageRole.USER, metadata["text"]))
            messages.append(yandexgpt.Message(yandexgpt.MessageRole.ASSISTANT, document))
    
    messages.append(yandexgpt.Message(yandexgpt.MessageRole.USER, question))
    return textGenerationAsync.sync_completion(messages, False, 0, 250, 20)["response"]["alternatives"][0]["message"]["text"]

@format_qa
def generate_chatgpt_answer(question: str):
    result = collection.query(
        query_embeddings=[embeddings.text_embedding(question)["embedding"]],
        n_results=6,
    )
    
    messages = [{"role": yandexgpt.MessageRole.SYSTEM.value, "content": SYSTEM_PROMPT}]
    for distance, metadata, document in zip(
            result["distances"][0], result["metadatas"][0], result["documents"][0]
    ):
        if distance < 1:
            messages.append({"role": yandexgpt.MessageRole.USER.value, "content": metadata["text"]})
            messages.append({"role": yandexgpt.MessageRole.ASSISTANT.value, "content": document})
    
    messages.append({"role": yandexgpt.MessageRole.USER.value, "content": question})
    return openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    ).choices[0].message.content

In [23]:
for question in questions:
    print("YandexGPT")
    generate_yandexgpt_answer(question)
    
    print("\n\n")
    print("ChatGPT 3.5")
    generate_chatgpt_answer(question)
    print("____________________________")

YandexGPT
Вопрос: Доброго времени! пытаюсь объединить товар . почему к данному товару не цепляется товар из яндекс маркета . штрихкод и артикул есть .
Ответ: Здравствуйте!

Чтобы разобраться в ситуации, мне нужно больше информации. Пожалуйста, опишите проблему более подробно:

- Что именно вы пытаетесь сделать? 
- Какой товар не получается объединить?
- Возникает ли какая-то ошибка при попытке объединения товаров?
- Если да, то укажите, пожалуйста, текст этой ошибки.

Также мне может потребоваться дополнительная информация или скриншоты, чтобы лучше понять ситуацию. Буду рад помочь вам!



ChatGPT 3.5
Вопрос: Доброго времени! пытаюсь объединить товар . почему к данному товару не цепляется товар из яндекс маркета . штрихкод и артикул есть .
Ответ: Для того чтобы товар из Яндекс.Маркета был связан с вашим товаром в SelSup, необходимо убедиться в следующем:

1. Убедитесь, что у товара в SelSup заполнены штрихкод и артикул точно так же, как и у товара в Яндекс.Маркете. Проверьте, что значе